# Session Summary Dashboard

Quick stats for a single day or a complete multi-day run.

## Data generation

```bash
# Single week
./build/qrsdp_run --seed 42 --days 5 --seconds 23400

# Full year
./build/qrsdp_run --seed 42 --days 252 --seconds 23400
```

In [7]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go

import qrsdp_reader as reader
import book_replay as replay

In [8]:
# --- Configuration ---
RUN_DIR = Path("../output/run_42")

manifest = reader.load_manifest(RUN_DIR)
symbols = reader.manifest_symbols(manifest)
multi_security = len(symbols) > 0

if multi_security:
    sessions = manifest["securities"][0]["sessions"]
    total_sessions = sum(len(s["sessions"]) for s in manifest["securities"])
    print(f"Run: {RUN_DIR.name}  (multi-security: {', '.join(symbols)})")
    print(f"Seed: {manifest['base_seed']}, total sessions: {total_sessions}")
else:
    sessions = manifest["sessions"]
    print(f"Run: {RUN_DIR.name}")
    print(f"Seed: {manifest['base_seed']}, days: {len(sessions)}")
print(f"Date range: {sessions[0]['date']} — {sessions[-1]['date']}")

Run: run_42
Seed: 42, days: 252
Date range: 2026-01-02 — 2026-12-21


## Single-Day Summary

In [9]:
DAY_INDEX = 0

session = sessions[DAY_INDEX]
day_file = RUN_DIR / session["file"]
header = reader.read_header(day_file)
events = reader.read_day(day_file)

file_size_bytes = os.path.getsize(day_file)
raw_size = len(events) * 26
compression_ratio = raw_size / file_size_bytes if file_size_bytes > 0 else 0

duration_s = (events["ts_ns"][-1] - events["ts_ns"][0]) / 1e9 if len(events) > 1 else 0
event_rate = len(events) / duration_s if duration_s > 0 else 0

# Book replay for price stats
book = replay.replay_book(
    events,
    p0_ticks=header["p0_ticks"],
    levels_per_side=header["levels_per_side"],
    initial_spread_ticks=header["initial_spread_ticks"],
    initial_depth=header["initial_depth"],
)

shifts = (np.diff(book["best_bid"]) != 0) | (np.diff(book["best_ask"]) != 0)
n_shifts = shifts.sum()

print(f"\n{'='*50}")
print(f"  Date:             {session['date']}")
print(f"  Events:           {len(events):>12,}")
print(f"  Duration:         {duration_s:>12,.1f} s")
print(f"  Event rate:       {event_rate:>12,.0f} events/s")
print(f"  Open price:       {book['mid_ticks'][0]:>12.1f} ticks")
print(f"  Close price:      {book['mid_ticks'][-1]:>12.1f} ticks")
print(f"  Max spread:       {book['spread_ticks'].max():>12d} ticks")
print(f"  Price shifts:     {n_shifts:>12,}")
print(f"  File size:        {file_size_bytes / 1024 / 1024:>12.2f} MB")
print(f"  Raw size:         {raw_size / 1024 / 1024:>12.2f} MB")
print(f"  Compression:      {compression_ratio:>12.2f}x")
print(f"{'='*50}")


  Date:             2026-01-02
  Events:              1,689,650
  Duration:             23,400.0 s
  Event rate:                 72 events/s
  Open price:            10000.0 ticks
  Close price:           10128.0 ticks
  Max spread:                  7 ticks
  Price shifts:          463,853
  File size:               21.02 MB
  Raw size:                41.90 MB
  Compression:              1.99x


### Event Type Breakdown

In [10]:
type_counts = np.bincount(events["type"], minlength=6)
breakdown = pd.DataFrame({
    "Event Type": [reader.EVENT_TYPES.get(i, f"UNKNOWN_{i}") for i in range(6)],
    "Count": type_counts[:6],
    "Percentage": [f"{100 * c / len(events):.1f}%" for c in type_counts[:6]],
})
breakdown

,Event Type,Count,Percentage
0,ADD_BID,388345,23.0%
1,ADD_ASK,388036,23.0%
2,CANCEL_BID,176072,10.4%
3,CANCEL_ASK,177093,10.5%
4,EXECUTE_BUY,280097,16.6%
5,EXECUTE_SELL,280007,16.6%


## Multi-Day Summary Table

In [11]:
def _summarise_sessions(session_list, label_prefix=""):
    """Build summary rows for a list of session dicts."""
    rows = []
    for session_info in session_list:
        fpath = RUN_DIR / session_info["file"]
        hdr = reader.read_header(fpath)
        evts = reader.read_day(fpath)
        fsize = os.path.getsize(fpath)

        bk = replay.replay_book(
            evts,
            p0_ticks=hdr["p0_ticks"],
            levels_per_side=hdr["levels_per_side"],
            initial_spread_ticks=hdr["initial_spread_ticks"],
            initial_depth=hdr["initial_depth"],
        )

        dur = (evts["ts_ns"][-1] - evts["ts_ns"][0]) / 1e9 if len(evts) > 1 else 0
        sh = ((np.diff(bk["best_bid"]) != 0) | (np.diff(bk["best_ask"]) != 0)).sum()

        row = {
            "Date": session_info["date"],
            "Events": f"{len(evts):,}",
            "Rate (ev/s)": f"{len(evts) / dur:,.0f}" if dur > 0 else "—",
            "Open": f"{bk['mid_ticks'][0]:.1f}",
            "Close": f"{bk['mid_ticks'][-1]:.1f}",
            "Shifts": f"{sh:,}",
            "File MB": f"{fsize / 1024 / 1024:.2f}",
            "Compression": f"{len(evts) * 26 / fsize:.2f}x" if fsize > 0 else "—",
        }
        if label_prefix:
            row["Symbol"] = label_prefix
        rows.append(row)
        print(f"  {label_prefix + ' ' if label_prefix else ''}"
              f"{session_info['date']}: {len(evts):>10,} events, close={bk['mid_ticks'][-1]:.1f}")
    return rows


if multi_security:
    rows = []
    for sec in manifest["securities"]:
        rows.extend(_summarise_sessions(sec["sessions"], label_prefix=sec["symbol"]))
    col_order = ["Symbol", "Date", "Events", "Rate (ev/s)", "Open", "Close",
                 "Shifts", "File MB", "Compression"]
else:
    rows = _summarise_sessions(sessions)
    col_order = ["Date", "Events", "Rate (ev/s)", "Open", "Close",
                 "Shifts", "File MB", "Compression"]

summary_df = pd.DataFrame(rows)[col_order]
summary_df

  2026-01-02:  1,689,650 events, close=10128.0
  2026-01-05:  1,686,335 events, close=9780.5
  2026-01-06:  1,688,569 events, close=9299.5
  2026-01-07:  1,687,213 events, close=8556.5
  2026-01-08:  1,688,827 events, close=8536.0
  2026-01-09:  1,686,873 events, close=8612.5
  2026-01-12:  1,688,922 events, close=7883.5
  2026-01-13:  1,688,382 events, close=7274.5
  2026-01-14:  1,684,628 events, close=7476.5
  2026-01-15:  1,687,903 events, close=7754.0
  2026-01-16:  1,687,343 events, close=7419.0
  2026-01-19:  1,684,937 events, close=7883.5
  2026-01-20:  1,687,952 events, close=7675.5
  2026-01-21:  1,688,439 events, close=8069.5
  2026-01-22:  1,688,152 events, close=7864.0
  2026-01-23:  1,687,788 events, close=7791.5
  2026-01-26:  1,689,577 events, close=8286.0
  2026-01-27:  1,685,813 events, close=7959.0
  2026-01-28:  1,686,369 events, close=8033.0
  2026-01-29:  1,687,068 events, close=7967.5
  2026-01-30:  1,686,470 events, close=8173.0
  2026-02-02:  1,685,445 events, 

,Date,Events,Rate (ev/s),Open,Close,Shifts,File MB,Compression
0,2026-01-02,"1,689,650",72,10000.0,10128.0,"463,853",21.02,1.99x
1,2026-01-05,"1,686,335",72,10127.5,9780.5,"462,571",20.98,1.99x
2,2026-01-06,"1,688,569",72,9780.5,9299.5,"462,121",21.00,1.99x
3,2026-01-07,"1,687,213",72,9299.0,8556.5,"462,982",20.99,1.99x
4,2026-01-08,"1,688,827",72,8555.5,8536.0,"464,086",21.01,1.99x
...,...,...,...,...,...,...,...,...
247,2026-12-15,"1,684,544",72,6131.5,6235.5,"461,983",20.95,1.99x
248,2026-12-16,"1,686,907",72,6234.5,6447.5,"462,493",20.98,1.99x
249,2026-12-17,"1,685,633",72,6446.5,6955.5,"464,154",20.97,1.99x
250,2026-12-18,"1,685,700",72,6955.0,6897.5,"462,732",20.97,1.99x


### Closing Price Across Days

In [12]:
fig_close = go.Figure()
colours = ["#1976D2", "#D32F2F", "#388E3C", "#F57C00", "#7B1FA2", "#0097A7"]

if multi_security:
    for idx, sym in enumerate(symbols):
        sym_rows = [r for r in rows if r.get("Symbol") == sym]
        fig_close.add_trace(go.Scatter(
            x=[r["Date"] for r in sym_rows],
            y=[float(r["Close"]) for r in sym_rows],
            mode="lines+markers", name=sym,
            line=dict(color=colours[idx % len(colours)], width=2),
            marker=dict(size=6),
        ))
else:
    fig_close.add_trace(go.Scatter(
        x=[r["Date"] for r in rows],
        y=[float(r["Close"]) for r in rows],
        mode="lines+markers", name="Close price",
        line=dict(color=colours[0], width=2),
        marker=dict(size=6),
    ))

fig_close.update_layout(
    title="Daily Closing Mid-Price",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price (ticks)"),
    height=400,
    template="plotly_white",
)
fig_close.show()